In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
import pickle

In [4]:
data = pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [5]:
data.drop(['RowNumber', 'CustomerId', 'Surname'], axis = 1, inplace = True)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
gender_le = LabelEncoder()
data['Gender'] = gender_le.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [7]:
data.shape

(10000, 11)

In [8]:
geography_ohe = OneHotEncoder()
geo_encoded = geography_ohe.fit_transform(data[['Geography']])
geo_encoded

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
geo_encoded.shape

(10000, 3)

In [10]:
geography_ohe.get_feature_names_out()

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
geo_encoded_df = pd.DataFrame(geo_encoded.toarray(), columns = geography_ohe.get_feature_names_out())
geo_encoded_df.head()

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0


In [12]:
data = pd.concat([data.drop('Geography', axis = 1), geo_encoded_df], axis = 1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
with open('gender_le.pkl', 'wb') as file:
    pickle.dump(gender_le, file)

with open('geography_ohe.pkl', 'wb') as file:
    pickle.dump(geography_ohe, file)

In [14]:
# Dependent and Independent Features

x = data.drop('Exited', axis = 1)
y = data['Exited']

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.23, random_state = 43)

In [15]:
scaler = StandardScaler()
x_train = scaler.fit_transform(x_train)
x_test = scaler.transform(x_test)

In [16]:
x_train

array([[ 0.22087277, -1.10873773, -0.47909843, ..., -0.99740596,
        -0.57795011,  1.72786204],
       [-1.93188539, -1.10873773, -0.38446838, ..., -0.99740596,
        -0.57795011,  1.72786204],
       [ 1.32830125, -1.10873773, -0.57372848, ...,  1.00260078,
        -0.57795011, -0.57874991],
       ...,
       [ 0.47961774, -1.10873773, -1.14150878, ..., -0.99740596,
         1.73025314, -0.57874991],
       [ 0.66591412,  0.90192655,  0.56183213, ..., -0.99740596,
        -0.57795011,  1.72786204],
       [-0.27591757,  0.90192655, -0.57372848, ..., -0.99740596,
         1.73025314, -0.57874991]], shape=(7700, 12))

In [17]:
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

------------------
ANN IMPLEMENTATION
------------------

In [18]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [31]:
model = Sequential([
    Dense(64, activation = 'relu', input_shape = (x_train.shape[1], )), # HL1 - connected with input layer
    Dense(32, activation = 'relu'), # HL2
    Dense(1, activation = 'sigmoid',) # output layer
])

d:\downloads\codes\DL project\ANN classification\venv\Lib\site-packages\keras\src\layers\core\dense.py:92: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [32]:
model.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_3 (Dense)                 │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,945 (11.50 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

In [33]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.01)
loss = tf.keras.losses.BinaryCrossentropy()
optimizer, loss

(<keras.src.optimizers.adam.Adam at 0x24add45b320>,
 <LossFunctionWrapper(<function binary_crossentropy at 0x0000024AD80411C0>, kwargs={'from_logits': False, 'label_smoothing': 0.0, 'axis': -1})>)

In [22]:
model.compile(optimizer = optimizer, loss = 'binary_crossentropy', metrics = ['accuracy'])

In [23]:
# setup the tensorboard
log_dir = 'logs/fit/' + datetime.datetime.now().strftime('%Y-%m-%d--%H-%M-%S')
tensorflow_callback = TensorBoard(log_dir = log_dir, histogram_freq = 1)

In [24]:
# setup earlystopping
early_stopping_callbacks = EarlyStopping(monitor = 'val_loss', patience = 15, restore_best_weights = True)

In [25]:
# training the model
history = model.fit(
    x_train, y_train, validation_data = (x_test, y_test), epochs = 100, callbacks = [tensorflow_callback, early_stopping_callbacks]
)

Epoch 1/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 6s 10ms/step - accuracy: 0.8394 - loss: 0.3945 - val_accuracy: 0.8557 - val_loss: 0.3650
Epoch 2/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8552 - loss: 0.3516 - val_accuracy: 0.8609 - val_loss: 0.3490
Epoch 3/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8579 - loss: 0.3492 - val_accuracy: 0.8609 - val_loss: 0.3467
Epoch 4/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 4ms/step - accuracy: 0.8594 - loss: 0.3422 - val_accuracy: 0.8565 - val_loss: 0.3549
Epoch 5/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - accuracy: 0.8655 - loss: 0.3366 - val_accuracy: 0.8570 - val_loss: 0.3535
Epoch 6/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8625 - loss: 0.3358 - val_accuracy: 0.8609 - val_loss: 0.3445
Epoch 7/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8648 - loss: 0.3310 - val_accuracy: 0.8604 - val_loss: 0.3589
Epoch 8/100
241/241 ━━━━━━━━━━━━━━━━━━━━ 1s 5ms/step - accuracy: 0.8651 - loss: 0.3306 - val_acc

In [26]:
model.save('model.h5')

In [35]:
# Load TensorBoard Extention
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [ ]:
# launch tensorboard session
%tensorboard --logdir logs/fit

In [ ]:
pip install tensorflow==2.20

2.20.0
